# Creating a Chatbot

In [ ]:
! pip install gradio vertexai

In [1]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, ChatSession


## Hello world with Gemini API
Answers are generated based on a question to Gemini.


In [2]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'path/to/your/credentials.json'



In [ ]:
model = GenerativeModel("gemini-2.0-flash-001")

question = 'What is the capital of Argentina?'
response = model.generate_content(question)

print(response.text)

'The capital of Argentina is Buenos Aires.'

## Chat conversations with Gemini
Gemini enables us to have a conversation. The ChatSession class simplifies the process by managing the state of the conversation, so unlike with generate_content, you do not have to store the conversation history as a list. With start_chat the history of the whole conversation is saved.

In [4]:
chat = model.start_chat(history=[])
countries = ['Chile', 'Bolivia']

for country in countries:
    message = f'What is the capital of {country}?'
    chat.send_message(message)

print(chat.history)

[role: "user"
 parts {
   text: "What is the capital of Chile?"
 },
 role: "model"
 parts {
   text: "The capital of Chile is Santiago. It is located in the central region of the country, in the Santiago Metropolitan Region. Santiago is the largest city in Chile, with a population of over 6 million people. The city is a major economic and cultural center for Chile, and it is home to many important historical and cultural landmarks. The city was founded in 1541 by Pedro de Valdivia, a Spanish conquistador. In 1810, Chile declared independence from Spain, and Santiago became the capital of the new Chilean Republic. Today, Santiago is a modern and cosmopolitan city that is home to a diverse population. The city is a popular tourist destination, and it is known for its beautiful architecture, its vibrant nightlife, and its many delicious restaurants."
 },
 role: "user"
 parts {
   text: "What is the capital of Bolivia?"
 },
 role: "model"
 parts {
   text: "The capital of Bolivia is Sucre.

You can also ask questions about the history of the conversation.


In [5]:
response = chat.send_message('From which countries did I ask you the capital city?')
print(response.text)

'You asked me two questions about capital cities. \r\n\r\nThe first question was: What is the capital of Chile?\r\n\r\nThe second question was: What is the capital of Bolivia?'

## Streaming Gradio Chatbots
Gradio is the quickest way to showcase your machine learning model through a user-friendly web interface, making it accessible to everyone, everywhere!

To develop chatbots, there are many ways to make a conversation in Gradio. In this tutorial, we will use gr.ChatInterface()

In [ ]:
import time
import gradio as gr

def slow_echo(message, history):
    for i in range(len(message)):
        time.sleep(0.3)
        yield "You typed: " + message[: i+1]

gr.ChatInterface(slow_echo).launch(share=True)

/Users/yassineabdennadher/miniconda3/envs/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


With that code, the system response is the same as the input text in the interface.

Finally, we will merge the Gemini and Gradio code to make a chatbot that responds to the input text. The code is edited to make the undo and clear buttons of the interface work.

**Note**: You should stop the cell just above to be able to run this one !

In [ ]:
import time
import gradio as gr
from vertexai.generative_models import Content, Part  # Import Part class as well

chat = model.start_chat(history=[])

def transform_history(history):
    new_history = []
    for user_input, model_response in history:
        new_history.append(Content(parts=[Part.from_text(user_input)], role="user"))
        new_history.append(Content(parts=[Part.from_text(model_response)], role="model"))
    return new_history

def response(message, history):
    global chat
    # The history will be the same as in Gradio, the 'Undo' and 'Clear' buttons will work correctly.
    chat = model.start_chat(history=transform_history(history))
    response = chat.send_message(message)

    # Each character of the answer is displayed
    for i in range(len(response.text)):
        time.sleep(0.05)
        yield response.text[: i+1]

gr.ChatInterface(response,
                 title='Gemini Chat',
                 textbox=gr.Textbox(placeholder="Question to Gemini")).launch(debug=True)

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


## Chatbot pizza

In [ ]:
import time
import gradio as gr
from vertexai.generative_models import Content, Part

# PizzaBot context message
system_message = """
You are PizzaBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collect the order, \
and then ask if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment. \
Make sure to clarify all options, extras, and sizes to uniquely \
identify the item from the menu. \
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""

# Initialize chat
chat = model.start_chat(history=[])

# Send the initial system message and get the initial response from the model
initial_message = Content(parts=[Part.from_text(system_message)], role="user")
initial_response = chat.send_message(system_message)

# Prepare the initial history
chat_history = [
    (system_message, initial_response.text)
]

def transform_history(history):
    new_history = []
    for user_input, model_response in history:
        new_history.append(Content(parts=[Part.from_text(user_input)], role="user"))
        new_history.append(Content(parts=[Part.from_text(model_response)], role="model"))
    return new_history

def response(message, history):
    global chat_history
    # Transform the history to include the initial system message and response
    transformed_history = transform_history(chat_history + history)
    
    # Initialize the chat with the current chat history
    chat = model.start_chat(history=transformed_history)
    
    # Send the user's message and get the response from the model
    model_response = chat.send_message(message)
    
    # Each character of the answer is displayed
    for i in range(len(model_response.text)):
        time.sleep(0.05)
        yield model_response.text[: i + 1]
    
    # Update the chat history with the latest user and model messages
    chat_history.append((message, model_response.text))

gr.ChatInterface(response,
                 title='PizzaBot Chat',
                 textbox=gr.Textbox(placeholder="Place your pizza order")).launch(debug=True)


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


## EXERCISE :

This a list of 5 IKEA tables with description, material, dimension.

Create a chatbot that accepts preferences of users that wish to buy a table and chooses one for them.


LISABO, Ash veneer, 140x78 cm
MÖRBYLÅNGA
Table, 220x100 cm
EKEDALEN
Extendable table, 180/240x90 cm
GUNNEBY
Table, 200x80 cm
DANDERYD
Dining table, 130x80 cm
TARSELE
Extendable table, 150/200x80 cm



What happens if you insult the chatbot?
What if you go off-topic?